## Based on CRTS_sel_g_cut_vs_r_cut.ipynb,   taking only the code needed to make Fig. 3

A testbed for making Fig.3 :  the 4 x 4 histogram plot. It shows that   0   < log(tau) < 1.7 (short timescales bin), does not exhibit any variability, because chi_ij for qso and stars is almost undistinguishable for uncorrected points. 

The files in  SF_plotting/Histogram_r_cut_starsB_qso/    are described in README. 

To make them  one would run points 1-10  (numeration inherited from CRTS_sel_g_cut_vs_r_cut.ipynb )   for different mag cuts :  17-18 ,  18-18.5 ,  18.5-19 , thus producing  8 files per cut (4 for qso,  4 for Blue Stars ) . 

Thus saved in SF_plotting/Histogram_r_cut_starsB_qso/   , one would read them all in 10)a) , and plot the histogram. 

NOTE:  that these points have no correction , they have exactly the same e_ij as from master files . |

NOTE : if Fig_3_prereq-get_tau_samples_and_fc.ipynb  has been executed, then we already have all the log(tau) samples, and so JUMP STRAIGHT TO PART 10)a) : plotting 


In [2]:
import os
import numpy as np 
import matplotlib.pyplot as plt
from astroML.stats import median_sigmaG
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import CRTS_paper_modules as mod
import datetime

In [32]:
cols1, qso_cat = mod.get_qso_catalog(catalog='DB_QSO') 
cols2 , star_cat= mod.get_stars_catalog()


Zipping CRTS-SDSS quasars catalog from  CRTS_SDSS_cross_matched_qso_DB_QSO_catalog.txt  ...
Read in  7601 , quasars from CRTS
zipping CRTS-SDSS stars catalog...


## 1) Read in points  m_ij , e_ij , tau  from master files 

In [34]:
Min = 18
Max = 18.5
magnitudes = ['r']  # don't need g magnitude , decided to use only r magnitude 

# Where to save the samples of log(tau), as well as the table results  ... 
outDir = os.path.join(os.getcwd(), 'Fig_3_data', datetime.datetime.now().strftime('%Y-%m-%d')+ '_histogram_table/')

# Create the dir if it does not yet exist ... 
if not os.path.exists(outDir): os.system('mkdir %s' % outDir)
print('Today we will save our Fig_3 files in %s'%outDir)

objects_in_cut = {}

for mag in magnitudes : 
    cut_mag = mag
   
    
    print('\nUsing now only lightcurves with SDSS  %f< %s < %f' % (Min, cut_mag, Max))


    good_ids_S_blue = mod.cut_stars(mMin = Min, mMax=Max, mErrMax = 0.3, gi_Min = -1,
                                              gi_Max=1, cut_mag=cut_mag + '_mMed')
    
    good_ids_S_red = mod.cut_stars(mMin = Min, mMax=Max, mErrMax = 0.3, gi_Min = 1, 
                                           gi_Max=3, cut_mag=cut_mag + '_mMed')
    
    good_ids_QSO = mod.cut_qso(mMin = Min, mMax=Max, mErrMax = 0.3, 
                                               cut_mag=cut_mag)
    objects_in_cut[mag] = {'starsB':good_ids_S_blue, 'starsR':good_ids_S_red, 
                           'qso':good_ids_QSO}
    


Using now only lightcurves with SDSS  18.000000< r < 18.500000

 Reporting SDSS r  

 These cuts reduced the number of stars  in the sample from 48250  to  1389

 These cuts reduced the number of stars  in the sample from 48250  to  2065

 These cuts reduced the number of qso  in the sample from 7601  to  333


## Since I'm only using r_cut,   I won't compare it with g_cut .  Thus making r_bin is the same as r_cut  

In [7]:
bins = {}
bin_types = ['r_cut']  # removed 'g_cut'

objects = objects_in_cut['r'].keys()

# first need to explicitly initialize the dictionaries 
for b in bin_types:
    bins[b] = {}
    
for obj in objects : 
    bins['r_cut'][obj] =  objects_in_cut['r'][obj]


## Read the xi, ei for objects in the r_cut ... 

In [9]:
inDirStars   = 'sf_file_per_LC/star/'
inDirQSO = 'sf_file_per_LC/qso/'

out_dic = {}

#for b in bins.keys():
# read in only r_cut 

b = 'r_cut'
print 'Reading in xi, ei for bin ', b
out_dic[b] = {}   # initialize the dic 

good_ids_S_blue = bins[b]['starsB']
good_ids_S_red = bins[b]['starsR']
good_ids_QSO = bins[b]['qso']

qso, starB, starR = mod.read_xi_ei(inDirStars, good_ids_S_blue, good_ids_S_red, inDirQSO,
              good_ids_QSO,xi_ei_data=None, fc=None)

# put into a dictionary : makes it more explicit 
out_dic[b] = {'starsB': starB, 'starsR': starR, 'qso':qso}


# Straight after reading-in xi, ei,   one can proceed directly to part 9) (one bin) or 10 : all bins sigma comparison 
# or to Saving just the log(tau) samples of xi, tau, ei. 


Reading in xi, ei for bin  r_cut
making new delflx, tau, xi arrays


----- Already read 98% of qso 

----- Already read 99% of Blue Stars 

----- Already read 100% of Red Stars returning xi, ei for ... 127 objects


## 10) Save to a file all points for samples of log(tau) for a given mag cut 

In [ ]:
## Save to a file points for log(tau) samples  for QSO, stars 

b = 'r_cut'   # or g_cut

tau_min_arr = [0,   2.3, 2.8, 3.2]
tau_max_arr = [1.7, 2.5, 3.0, 3.4]
for obj in ['qso','starsB']:
    #obj = 'qso'  # or starsB,  starsR 
    m_ij = out_dic[b][obj][0]
    tau =  out_dic[b][obj][1]
    e_ij =  out_dic[b][obj][2]
    
    for i in range(len(tau_min_arr)):
        m1 = tau_min_arr[i] < np.log10(tau)
        m2 = np.log10(tau) < tau_max_arr[i]
        mask =  m1 * m2 

        data = np.column_stack((m_ij[mask], tau[mask], e_ij[mask]))
        figtitle = b+'_'+str(Min)+'-'+str(Max)+'_'+obj+'_fc-'+str(fc)+'_SF.png'

        fname = b+'_'+str(Min)+'-'+str(Max)+'_'+obj+'_mi_tau_ei-log_tau_'+\
                str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'.txt'
        print 'Saved', fname
        np.savetxt(outDir+fname, data, fmt='%s', delimiter= ' ')


## 10)a) Plot the histogram grid for qso and starsB 

In [17]:
# this is taken from poster_hist_r_cut_qso_starsB_mag_tau_grid.py  program

b = 'r_cut'
# Use today's data... 
#outDir = os.path.join(os.getcwd(), 'Fig_3_data', datetime.datetime.now().strftime('%Y-%m-%d')+ '_histogram_table/')

# Use data from another day 
outDir = os.path.join(os.getcwd(), 'Fig_3_data','2016-02-12_histogram_table/')

def plot2Chistograms(chiQSO, chiSTAR, Xmin, Xmax, Ymin, Ymax, Xlabel, Ylabel, ax, bins=20, title=''):
    limits = [(Xmin, Xmax, Ymin, Ymax)]
    labels = [Xlabel, Ylabel]
    ax.set_xlim(Xmin, Xmax)
    ax.set_ylim(Ymin, Ymax)
    #ax.set_xlabel(Xlabel, fontsize=12)
    #ax.set_ylabel(Ylabel, fontsize=12)
     # http://stackoverflow.com/questions/16150819/common-xlabel-ylabel-for-matplotlib-subplots
        
        
    #plt.tick_params(axis='both', which='major', labelsize=15)
    xTitle = Xmin + 0.05*(Xmax-Xmin)
    yTitle = Ymax - 0.2*(Ymax-Ymin)
    ax.text(xTitle, yTitle, title, fontsize=12)

    # plot a histogram
    ax.hist(chiSTAR, bins=bins, normed=True, facecolor='blue', histtype='stepfilled', alpha=0.4)
    ax.hist(chiQSO, bins=bins, normed=True, facecolor='red', histtype='stepfilled', alpha=0.2)

    # plot the robust width of both distributions
    stdev_rob_QSO = 0.7414 *(np.percentile(chiQSO,75) - np.percentile(chiQSO,25) )
    stdev_rob_S = 0.7414 *(np.percentile(chiSTAR,75) - np.percentile(chiSTAR,25) )
    
    #al_Q = 0.4
    #al_S = 0.2
    #s = '--'
    #w = 1.5
    #c = 0.5
    # below ymin, ymax in the absolute 0-1 units!  
    #ax.axvline(x =stdev_rob_S ,ymin=0, ymax=c, color='red', lw=w, ls=s, alpha=al_S)
    #ax.axvline(x =stdev_rob_S ,ymin=0, ymax=c, color='red', lw=w, ls=s, alpha=al_S)
    #ax.axvline(x =stdev_rob_QSO ,ymin=0, ymax=c, color='blue', lw=w, ls=s, alpha=al_Q)
    #ax.axvline(x =stdev_rob_QSO , ymin=0, ymax=c, color='blue', lw=w, ls=s, alpha=al_Q)
    #ax.axvline(x =-stdev_rob_S , ymin=0, ymax=c,color='red', lw=w, ls=s, alpha=al_S)
    #ax.axvline(x =-stdev_rob_S , ymin=0, ymax=c,color='red', lw=w, ls=s, alpha=al_S)
    #ax.axvline(x =-stdev_rob_QSO ,ymin=0, ymax=c, color='blue', lw=w, ls=s, alpha=al_Q)
    #ax.axvline(x =-stdev_rob_QSO , ymin=0, ymax=c,color='blue', lw=w, ls=s, alpha=al_Q)
   
    xTitle = Xmin + 0.65*(Xmax-Xmin)
    yTitle = Ymax - 0.2*(Ymax-Ymin)
    StarSigmaG = r'$'+str(stdev_rob_S)[:4]+'$'
    ax.text(xTitle, yTitle, StarSigmaG, fontsize=12)
    
    
    xTitle = Xmin + 0.65*(Xmax-Xmin)
    yTitle = Ymax - 0.35*(Ymax-Ymin)
    QSOSigmaG = r'$'+str(stdev_rob_QSO)[:4]+'$'
    ax.text(xTitle, yTitle, QSOSigmaG, fontsize=12)
    
    
Min_arr = [17, 18, 18.5]
Max_arr = [18, 18.5, 19]
tau_min_arr = [0,   2.3, 2.8, 3.2]
tau_max_arr = [1.7, 2.5, 3.0, 3.4]
xlims_arr = [5,10,10,10]




In [9]:
# just for testing  - load only 1/16 files (one for QSO one for Stars) and experiment with how it looks 
# if you use that, make sure to comment out those lines below !  
j=0
i=0

datafileS = outDir+b+'_'+str(Min_arr[j])+'-'+str(Max_arr[j])+'_'+'starsB'+'_mi_tau_ei-log_tau_'+\
                        str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'.txt'
vS = np.loadtxt(datafileS, unpack=True)

datafileQ = outDir+b+'_'+str(Min_arr[j])+'-'+str(Max_arr[j])+'_'+'qso'+'_mi_tau_ei-log_tau_'+\
        str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'.txt'
vQ = np.loadtxt(datafileQ, unpack=True)

        

In [28]:
import matplotlib 
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 

fig, axs = plt.subplots(4,3, figsize=(8, 8))
fig.subplots_adjust(wspace=0.46, hspace=0.36, left=0.12, right=0.94, bottom=0.10, top=0.95)

for i in range(len(tau_max_arr)):  # 
    for j in range(len(Min_arr) ):  # 
       

        #datafileS = outDir+b+'_'+str(Min_arr[j])+'-'+str(Max_arr[j])+'_'+'starsB'+'_mi_tau_ei-log_tau_'+\
        #                str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'.txt'
        #vS = np.loadtxt(datafileS, unpack=True)
        chiS = vS[0]/vS[2]
        chiSok = chiS[np.abs(chiS)<5]
        
        #datafileQ = outDir+b+'_'+str(Min_arr[j])+'-'+str(Max_arr[j])+'_'+'qso'+'_mi_tau_ei-log_tau_'+\
        #        str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'.txt'
        #vQ = np.loadtxt(datafileQ, unpack=True)
        chiQ = vQ[0]/vQ[2]
        chiQok = chiQ[np.abs(chiQ)<xlims_arr[i]]

        # plot histograms
        Xlabel = '$\chi = \Delta mag / error$'
        Ylabel = '$n / (N\Delta_{bin})$'
        Xmin = -xlims_arr[i]
        Xmax = xlims_arr[i]
        bins = 40 
        title= r'$ '+' '+ str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'$'
        plot2Chistograms(chiQok, chiSok, Xmin=Xmin, Xmax=Xmax, Ymin=0.0, 
                             Ymax=0.55, Xlabel=Xlabel, Ylabel=Ylabel, ax=axs[i,j],bins=bins,  
                         title=title)
fig.text(0.5, 0.04, Xlabel, ha='center', va='center',fontsize=20)
fig.text(0.03, 0.5, Ylabel, ha='center', va='center', rotation='vertical',fontsize=20)

#name = 'poster_r_cut_qso_starsB_histogram_grid.png'
name = outDir+'Fig_3_histogram_panels.png'
if (name is None):
    plt.show() 
else:
    print 'saving plot to:', name
    plt.savefig(name, bbox_inches='tight')

saving plot to: /astro/users/suberlak/Desktop/SDSS/SF_plotting/Fig_3_data/2016-02-12_histogram_table/Fig_3_histogram_panels.png


In [29]:
plt.show()